In [ ]:
#This experiment to evaluate the effect of spectral
#features on detection task for each disfluency
#we will use MFCC+ASR+TD+att 10 experiments
#----------------------------------------------
import tensorflow as tf
from tensorflow.keras.layers import Permute,Input,add,GlobalAveragePooling1D,concatenate,Concatenate,MaxPooling1D,Bidirectional,Flatten,MaxPooling2D,GlobalAveragePooling2D,Lambda,Dropout,BatchNormalization,Dense,TimeDistributed,LayerNormalization,Reshape,LSTM,Conv1D,Conv2D,Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler,ModelCheckpoint

from keras import backend as K
from keras.models import Model, load_model
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau
from keras.utils.vis_utils import plot_model

from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score
import librosa
import matplotlib.pyplot as plt
import librosa.display

import pandas as pd
import numpy as np

#----------------------------------------------

In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
test_path="/content/drive/MyDrive/classification_model/Thesis_experiments/extra_experiments/optimized_model/data/test_fb_new.csv"
train_path="/content/drive/MyDrive/classification_model/Thesis_experiments/extra_experiments/optimized_model/data/train_fb_new.csv"


test=pd.read_csv(test_path)
test = test.iloc[1: , :]

train=pd.read_csv(train_path)
train = train.iloc[1: , :]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test.head(50)

In [ ]:
# # Drop first column of dataframe
# test = test.iloc[: , 1:]

# # Drop first column of dataframe
# train = train.iloc[: , 1:]

In [ ]:
test

In [ ]:
y_train=train.iloc[:,-5:]
# # y_train=train.iloc[:,-5:-4]
# y_train=train.iloc[:,-1:]

X_train=train.drop(train.iloc[:,-8:],axis=1)

y_test=test.iloc[:,-5:]
# # y_test=test.iloc[:,-5:-4]
# y_test=test.iloc[:,-1:]

X_test=test.drop(test.iloc[:,-8:],axis=1)

In [ ]:
#This Blcok to determine the stuttering events in our experiment
#By Kareem
#----------------------------------------------
disfluencies=['Prolongation','Block','SoundRep','WordRep','Interjection','NoStutteredWords']
# disfluencies=['Prolongation','Block','SoundRep','WordRep','Interjection']
# disfluencies=['Stutter','Normal']
[disfluency for disfluency in disfluencies]
#----------------------------------------------

In [ ]:
df_train=pd.DataFrame(X_train)
df_test=pd.DataFrame(X_test)

In [ ]:
df_test

In [ ]:


X_train_asr=df_train.iloc[:,-4768:]
df_train.drop(df_train.iloc[:,-4768:],axis=1,inplace=True)

X_test_asr=df_test.iloc[:,-4768:]
df_test.drop(df_test.iloc[:,-4768:],axis=1,inplace=True)




In [ ]:
X_train_ff=df_train.iloc[:,-903:]
df_train.drop(df_train.iloc[:,-903:],axis=1,inplace=True)

X_test_ff=df_test.iloc[:,-903:]
df_test.drop(df_test.iloc[:,-903:],axis=1,inplace=True)

In [ ]:
X_train_ac=df_train.iloc[:,-300:]
df_train.drop(df_train.iloc[:,-300:],axis=1,inplace=True)

X_test_ac=df_test.iloc[:,-300:]
df_test.drop(df_test.iloc[:,-300:],axis=1,inplace=True)

In [ ]:
X_train_zc=df_train.iloc[:,-301:]
df_train.drop(df_train.iloc[:,-301:],axis=1,inplace=True)

X_test_zc=df_test.iloc[:,-301:]
df_test.drop(df_test.iloc[:,-301:],axis=1,inplace=True)

In [ ]:
X_train_mfcc_2d=df_train
X_test_mfcc_2d=df_test

In [ ]:
X_test_mfcc_2d

In [ ]:
X_test_ff

In [ ]:
y_train=y_train
y_train = y_train.astype(int)

y_test=y_test
y_test = y_test.astype(int)

In [ ]:
y_test

In [ ]:
y_test

In [ ]:
import seaborn as sns
categories = list(y_test.columns.values)
sns.set(font_scale = 2)
plt.figure(figsize=(15,9))
ax= sns.barplot(categories, y_test.iloc[:,:].sum().values)

plt.title("Disfluency in each category", fontsize=24)
plt.ylabel('Number of Disfluencies', fontsize=18)
plt.xlabel('Disfluency Type ', fontsize=18)
#adding the text labels
rects = ax.patches
labels = y_test.iloc[:,0:].sum().values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=18)
plt.show()

In [ ]:
x_train_asr=np.array(X_train_asr).reshape(len(X_train_asr),149,32)
x_valid_asr=np.array(X_test_asr).reshape(len(X_test_asr),149,32)



x_train_ff=np.array(X_train_ff).reshape(len(X_train_ff),3,301)
x_valid_ff=np.array(X_test_ff).reshape(len(X_test_ff),3,301)

x_train_ac=np.array(X_train_ac).reshape(len(X_train_ac),1,300)
x_valid_ac=np.array(X_test_ac).reshape(len(X_test_ac),1,300)

x_train_zc=np.array(X_train_zc).reshape(len(X_train_zc),1,301)
x_valid_zc=np.array(X_test_zc).reshape(len(X_test_zc),1,301)

x_train_mfcc_2d=np.array(X_train_mfcc_2d).reshape(len(X_train_mfcc_2d),20,301)
x_valid_mfcc_2d=np.array(X_test_mfcc_2d).reshape(len(X_test_mfcc_2d),20,301)

# Y_train=np.array(y_train)
# y_valid=np.array(y_test)
# Y=Y_train
# y_test=y_valid



X_mfcc_2d=x_train_mfcc_2d
X_asr=x_train_asr

X_ff=x_train_ff
X_ac=x_train_ac
X_zc=x_train_zc




In [ ]:

spect = X_train_mfcc_2d.iloc[9]

spect=np.array(spect).reshape(20,301)

print(spect.shape)
plt.figure(figsize=(10, 4))
librosa.display.specshow(spect, y_axis='mel', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.show()

In [ ]:
model_paramerters={
    'no_of_frames':40,
    'no_of_classes':1,
    'frequency':1,
    'no_channels':1,
    'no_conv_layers':1,
    'conv':{
        'kernal_size':(3,3),
        'kernal_size_1d':3,
        'padding':'same',
        'activation':'relu'

    },
    'lstm':
    {

    }


}

def custom_f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))

        recall = TP / (Positives+K.epsilon())
        return recall


    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

        precision = TP / (Pred_Positives+K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)

    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [ ]:
#https://github.com/kobiso/CBAM-keras/blob/master/models/attention_module.py
class ChannelAttention(tf.keras.layers.Layer):
      def __init__(self, filters, ratio):
        super(ChannelAttention, self).__init__()
        self.filters = filters
        self.ratio = ratio

        def build(self, input_shape):
            self.shared_layer_one = tf.keras.layers.Dense(self.filters//self.ratio,
                             activation='relu', kernel_initializer='he_normal',
                              use_bias=True,
                              bias_initializer='zeros')
            self.shared_layer_two = tf.keras.layers.Dense(self.filters,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

        def call(self, inputs):
            # AvgPool
            avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inputs)


            avg_pool = self.shared_layer_one(avg_pool)
            avg_pool = self.shared_layer_two(avg_pool)

            # MaxPool
            max_pool = tf.keras.layers.GlobalMaxPooling1D()(inputs)
            max_pool = tf.keras.layers.Reshape((1,1,filters))(max_pool)

            max_pool = shared_layer_one(max_pool)
            max_pool = shared_layer_two(max_pool)


            attention = tf.keras.layers.Add()([avg_pool,max_pool])
            attention = tf.keras.layers.Activation('sigmoid')(attention)

            return tf.keras.layers.Multiply()([inputs, attention])


class SpatialAttention(tf.keras.layers.Layer):
      def __init__(self, kernel_size):
        super(SpatialAttention, self).__init__()
        self.kernel_size = kernel_size

        def build(self, input_shape):
            self.conv2d = tf.keras.layers.Conv2D(filters = 1,
                    kernel_size=self.kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)

        def call(self, inputs):

            # AvgPool
            avg_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=3, keepdims=True))(inputs)

            # MaxPool
            max_pool = tf.keras.layers.Lambda(lambda x: tf.keras.backend.max(x, axis=3, keepdims=True))(inputs)

            attention = tf.keras.layers.Concatenate(axis=3)([avg_pool, max_pool])

            attention = self.conv2d(attention)


            return tf.keras.layers.multiply([inputs, attention])

In [ ]:
def cnn_rnn_1d(x,no_filters,k_size,activation_function):
  x=tf.keras.layers.Conv1D(no_filters, kernel_size =3 ,strides=1, padding='same',activation=activation_function
                             )(x)

  x=tf.keras.layers.BatchNormalization()(x, training=True)
  out=tf.keras.layers.Dropout(0.1)(x)
  return out

def cnn_rnn_2d(x,no_filters,k_size,activation_function):
  x=tf.keras.layers.Conv2D(no_filters, kernel_size =3 ,strides=1, padding='same',activation=activation_function
                            )(x)

  x=tf.keras.layers.BatchNormalization()(x, training=True)
  out=tf.keras.layers.Dropout(0.1)(x)
  return out

In [ ]:



av=[]
def proposed_model_build():
  print('Generate disfluency model...')

  input_asr=tf.keras.layers.Input(shape=(149,32))



  input_ff=tf.keras.layers.Input(shape=(3,301))
  input_ac=tf.keras.layers.Input(shape=(1,300))
  input_zc=tf.keras.layers.Input(shape=(1,301))
  input_mfcc_2d=tf.keras.layers.Input(shape=(20,301,1))



  x_asr=cnn_rnn_1d(input_asr,256,(3,3),'relu')
  x_asr=ChannelAttention(256, 8)( x_asr)
  x_asr=SpatialAttention(7)(x_asr)


  x_ff=cnn_rnn_1d(input_ff,256,(3,3),'relu')
  x_ff=ChannelAttention(256, 8)( x_ff)
  x_ff=SpatialAttention(7)(x_ff)

  x_ac=cnn_rnn_1d(input_ac,256,(3,3),'relu')
  x_ac=ChannelAttention(256, 8)( x_ac)
  x_ac=SpatialAttention(7)(x_ac)

  x_zc=cnn_rnn_1d(input_zc,256,(3,3),'relu')
  x_zc=ChannelAttention(256, 8)( x_zc)
  x_zc=SpatialAttention(7)(x_zc)

  x_mfcc_2d=cnn_rnn_2d(input_mfcc_2d,256,(3,3),'relu')
  x_mfcc_2d=ChannelAttention(256, 8)( x_mfcc_2d)
  x_mfcc_2d=SpatialAttention(7)(x_mfcc_2d)



  #------------- ASR-----------#

  lstm_asr = Bidirectional(LSTM(64,return_sequences=True),name="wav2vec_bi")(x_asr)
  lstm_asr=TimeDistributed(Dense(149),name="wav2vec_time")(lstm_asr)

  lstm_asr=tf.keras.layers.BatchNormalization()(lstm_asr, training=True)
  lstm_asr = Dropout(0.4)(lstm_asr)


  lstm_asr=GlobalAveragePooling1D()(lstm_asr)
  # #------------- ASR-----------#


  #------------- ff-----------#

  lstm_ff = Bidirectional(LSTM(64,return_sequences=True),name="ff_bi")(x_ff)
  lstm_ff=TimeDistributed(Dense(64),name="ff_time")(lstm_ff)

  lstm_ff=tf.keras.layers.BatchNormalization()(lstm_ff, training=True)
  lstm_ff = Dropout(0.4)(lstm_ff)

  lstm_ff=GlobalAveragePooling1D()(lstm_ff)
  # #------------- ff-----------#

   #------------- ac-----------#

  lstm_ac = Bidirectional(LSTM(64,return_sequences=True),name="ac_bi")(x_ac)
  lstm_ac=TimeDistributed(Dense(64),name="ac_time")(lstm_ac)

  lstm_ac=tf.keras.layers.BatchNormalization()(lstm_ac, training=True)
  lstm_ac = Dropout(0.4)(lstm_ac)

  lstm_ac=GlobalAveragePooling1D()(lstm_ac)
  # #------------- ac-----------#

  lstm_zc = Bidirectional(LSTM(64,return_sequences=True),name="zc_bi")(x_zc)
  lstm_zc=TimeDistributed(Dense(64),name="zc_time")(lstm_zc)

  lstm_zc=tf.keras.layers.BatchNormalization()(lstm_zc, training=True)
  lstm_zc = Dropout(0.4)(lstm_zc)

  lstm_zc=GlobalAveragePooling1D()(lstm_zc)
  # lstm_zc = Dense(128, activation='relu')(lstm_zc)
  # #------------- ac-----------#

  #------------- RCNN-----------#
  lstm_mfcc_2d = Permute((2, 1, 3))(x_mfcc_2d)
  # lstm_mfcc_2d = Permute((2, 1, 3))(x_mfcc_2d)
  lstm_mfcc_2d = Reshape((input_mfcc_2d.shape[-2], -1))(lstm_mfcc_2d)
  lstm_mfcc_2d = Bidirectional(LSTM(64,return_sequences=True))(lstm_mfcc_2d)
  lstm_mfcc_2d=TimeDistributed(Dense(64))(lstm_mfcc_2d)

  lstm_mfcc_2d=tf.keras.layers.BatchNormalization()(lstm_mfcc_2d, training=True)
  lstm_mfcc_2d = Dropout(0.4)(lstm_mfcc_2d)
  # attention_layer = ChannelAttention(47, 64)(lstm_mfcc_2d)

  lstm_mfcc_2d=GlobalAveragePooling1D()(lstm_mfcc_2d)
  concat = concatenate([lstm_mfcc_2d,lstm_asr,lstm_ff,lstm_ac,lstm_zc], axis=-1, name ='concat')


  out = Dense(model_paramerters['no_of_classes'], activation='sigmoid')(concat)
  # model = tf.keras.models.Model(inputs = [input_mfcc_2d,input_asr,input_ff], outputs = out)
  model = tf.keras.models.Model(inputs = [input_mfcc_2d,input_asr,input_ff,input_ac,input_zc], outputs = out)
  # model = tf.keras.models.Model(inputs = [input_mfcc_2d,input_asr,input_ff,input_ac], outputs = out)
  return model

In [ ]:
model=proposed_model_build()
plot_model(model, show_shapes=True, to_file='model.png')
model.summary()

In [ ]:
class WeightedAverage(tf.keras.layers.Layer):

    def __init__(self, n_output):
        super(WeightedAverage, self).__init__()
        self.W = tf.Variable(initial_value=tf.random.uniform(shape=[1,1,n_output], minval=0, maxval=1),
            trainable=True) # (1,1,n_inputs)

    def call(self, inputs):

        # inputs is a list of tensor of shape [(n_batch, n_feat), ..., (n_batch, n_feat)]
        # expand last dim of each input passed [(n_batch, n_feat, 1), ..., (n_batch, n_feat, 1)]
        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = Concatenate(axis=-1)(inputs) # (n_batch, n_feat, n_inputs)
        weights = tf.nn.softmax(self.W, axis=-1) # (1,1,n_inputs)
        # weights sum up to one on last dim

        return tf.reduce_sum(weights*inputs, axis=-1) # (n_batch, n_feat)

In [ ]:
!rm *.hdf5
early_stopping=EarlyStopping(monitor='val_loss', patience=5, verbose=1, min_delta=1e-4)

rm: cannot remove '*.hdf5': No such file or directory


In [ ]:
y_test=test.iloc[:,-5:]
y_test['I']

In [ ]:
test_temp=test
train_temp=train

In [ ]:
#kfold = KFold(n_splits=10, shuffle=True)
results=[]
labels=['P','B','S','W','I']

for exp in range(0,5):
  y_test=test_temp.iloc[:,-5:]
  Y_train=train_temp.iloc[:,-5:]
  index_test=labels[exp]
  index_train=labels[exp]
  y_tes=y_test[index_test]
  # print(y_tes)
  Y_tra=Y_train[index_train]


  Y_train=np.array(Y_tra)
  y_valid=np.array(y_tes)
  Y=Y_train
  y_test=y_valid
  print(Y)
  class_weights=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(Y),y=Y)
  class_weights={0:class_weights[0],1:class_weights[1]}
  print(class_weights)



  kfold = KFold(n_splits=10, shuffle=True,random_state=1)
  i=0
  # model =proposed_model_build()
  for train, test in kfold.split(X_mfcc_2d, Y):
    i=i+1
    print("Experiment ",exp,"Fold ",i)
    model =proposed_model_build()
    opt = tf.optimizers.Adam(learning_rate=0.0001)
    # metrics=[custom_f1]
    # model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
    model.compile(optimizer=opt,loss=tfa.losses.SigmoidFocalCrossEntropy(),metrics=['accuracy'])
    chk_point=ModelCheckpoint(filepath='/content/drive/MyDrive/classification_model/Thesis_experiments/extra_experiments/optimized_model/results/fb_all/models/mfcc2d-model-Exp-'+str(index_train)+"-Fold-"+str(i)+'.hdf5', verbose=1,
                            save_best_only=True, save_weights_only=True, mode='auto')
    # 256
    # Focal loss

    # history=model.fit([X_mfcc_2d[train],X_asr[train],X_ff[train],X_ac[train]],Y[train],epochs=300, batch_size=256,verbose=1,validation_data=([X_mfcc_2d[test],X_asr[test],X_ff[test],X_ac[test]],Y[test] ),callbacks=[early_stopping,chk_point])
    # history=model.fit(X_mfcc_2d[train],Y[train],epochs=300, batch_size=256,verbose=1,validation_data=([X_mfcc_2d[test]],Y[test] ),callbacks=[early_stopping,chk_point])
    # history=model.fit([X_mfcc_2d[train],X_asr[train],X_ff[train]],Y[train],epochs=300, batch_size=256,verbose=1,validation_data=([X_mfcc_2d[test],X_asr[test],X_ff[test] ],Y[test]),callbacks=[early_stopping,chk_point])
    history=model.fit([X_mfcc_2d[train],X_asr[train],X_ff[train],X_ac[train],X_zc[train]],Y[train],epochs=300, batch_size=128,verbose=1,validation_data=([X_mfcc_2d[test],X_asr[test],X_ff[test],X_ac[test],X_zc[test]],Y[test] ),callbacks=[early_stopping,chk_point])


    t_loss_values = history.history['loss']
    epochs = range(1, len(t_loss_values)+1)
    loss_values = history.history['val_loss']

    plt.plot(epochs, t_loss_values, label='Training Loss')
    plt.plot(epochs, loss_values, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()
    # scores = model.evaluate([X_mfcc_2d[test]], Y[test], verbose=1)
    # scores = model.evaluate([X_mfcc_2d[test],X_asr[test],X_ff[test]], Y[test], verbose=1)
    scores = model.evaluate([X_mfcc_2d[test],X_asr[test],X_ff[test],X_ac[test],X_zc[test]], Y[test], verbose=1)
    results.append(scores[1]*100)
    # print(" Kareem: %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("*"*100)




In [ ]:
y_test

In [ ]:
all_true_list=[]
opt = tf.optimizers.Adam(learning_rate=0.0001)
target_names = ['P','B','S','W','I']

# target_names = ['S','W','I']
X_asr=np.array(X_test_asr).reshape(len(X_test_asr),149,32)

X_ff=np.array(X_test_ff).reshape(len(X_test_ff),3,301)
X_ac=np.array(X_test_ac).reshape(len(X_test_ac),1,300)
X_zc=np.array(X_test_zc).reshape(len(X_test_zc),1,301)
X_mfcc_2d=np.array(X_test_mfcc_2d).reshape(len(X_test_mfcc_2d),20,301)







Y=y_test

f1_avg=[[],[],[],[],[]]
recall_avg=[[],[],[],[],[]]
acc_avg=[[],[],[],[],[]]
precision_avg=[[],[],[],[],[]]
df_result=pd.DataFrame()

from sklearn.metrics import f1_score,recall_score,average_precision_score
from tqdm import tqdm
for exp in range(0,5):
  y_test=test_temp.iloc[:,-5:]
  y_tes=y_test[target_names[exp]]
  y_test = y_tes.astype(int)

  for i in range(1,11):
      print("Results for : ",target_names[exp])
      model.load_weights(filepath='/content/drive/MyDrive/classification_model/Thesis_experiments/extra_experiments/optimized_model/results/fb_all/models/mfcc2d-model-Exp-'+target_names[exp]+"-Fold-"+str(i)+'.hdf5')
      model.compile(optimizer=opt,loss=tfa.losses.SigmoidFocalCrossEntropy(),metrics=['accuracy'])

      # y_scores = model.predict([X_asr])
      # y_scores = model.predict([X_mfcc_2d,X_asr,X_ff,X_ac,X_zc])
      # y_scores = model.predict([X_mfcc,X_asr,X_ff])
      y_scores = model.predict([X_mfcc_2d,X_asr,X_ff,X_ac,X_zc])
      y_pred=[]
      for sample in  y_scores:
        y_pred.append([1 if i>=0.5 else 0 for i in sample ] )
      y_pred = np.array(y_pred)


      label_names = ['Prolongation', 'Block', 'Sound', 'Word','Interjection','Fluent']
      matrix=confusion_matrix(y_test, y_pred)
      tp=matrix[0][0]
      tn=matrix[1][1]
      fp=matrix[1][0]
      fn=matrix[0][1]
      accuracy=(matrix[0][0]+matrix[1][1])/831

      f1_macro=f1_score(y_test, y_pred, average='macro')
      f1_weighted=f1_score(y_test, y_pred, average='weighted')
      f1_micro=f1_score(y_test, y_pred, average='micro')

      recall_macro=recall_score(y_test, y_pred, average='macro')
      recall_weighted=recall_score(y_test, y_pred, average='weighted')
      recall_micro=recall_score(y_test, y_pred, average='micro')


      pre_macro=precision_score(y_test, y_pred,average='macro')
      pre_weighted=precision_score(y_test, y_pred, average='weighted')
      pre_micro=precision_score(y_test, y_pred, average='micro')



      print(accuracy,f1_weighted,recall_weighted,pre_weighted)

      print(matrix)
      print(classification_report(y_test, y_pred))
      s = pd.Series([label_names[exp],tp,tn,fp,fn,accuracy,f1_macro,f1_weighted,f1_micro,recall_macro,recall_weighted,recall_micro,pre_macro,pre_weighted,pre_micro])

      df_result= df_result.append(s ,ignore_index=True)



In [ ]:
header=[label_names[exp],"tp","tn","fp","fn","accuracy","f1_macro","f1_weighted","f1_micro","recall_macro","recall_weighted","recall_micro","pre_macro","pre_weighted","pre_micro"]
df_result.columns =header
df_result

In [ ]:
df_result.to_csv("/content/drive/MyDrive/classification_model/Thesis_experiments/extra_experiments/optimized_model/results/csv/fb_all.csv")